In [ ]:
# Get the values below from the CDK deploy output.
%profile glue
%idle_timeout 2880
%glue_version 4.0
%worker_type Standard
%number_of_workers 1
%list_sessions
%status
%connections EwstackGlueConnectionAB87C2-HuJWcj6VPADi
%iam_role arn:aws:iam::400678530796:role/ew-edgewalker-dev-JobVerticesGlueRole1B693BCC-3MmteKZOfrjg
%%configure
{
  "--session-language": "python",
  "--job-language": "python",
  "--continuous-log-logGroup": "ew-edgewalker-dev-JobVerticesJobVerticesjobvertices95F1CA39-8RrrssSeE6mH",
  "--enable-continuous-cloudwatch-log": "true",
  "--enable-continuous-log-filter" : "true",
  #"--enable-metrics": "",
  "--additional-python-modules": "awswrangler==3.9.1,awswrangler[gremlin],SPARQLWrapper,requests"
}

In [ ]:
%reconnect 628c0a85-2f98-44d4-9523-8ece03d3781d

In [ ]:
%stop_session

In [ ]:
work_bucket = "ew-edgewalker-dev-workbucketf885502b-cibicr8xgwo0"
neptune_hostname = "neptunedbcluster-aa9odmqjepom.cluster-c9v5npv1veav.us-east-1.neptune.amazonaws.com"
neptune_role = "arn:aws:iam::400678530796:role/ew-edgewalker-dev-CommonCrawlNeptuneRole25077696-8NhOKWP86bRN"

In [ ]:
import boto3
import awswrangler as wr

In [ ]:
s3_client = boto3.client('s3', 'us-east-1')
objs = s3_client.list_objects(
    Bucket='commoncrawl',
    Prefix='projects/hyperlinkgraph/cc-main-2024-jun-jul-aug/host/vertices'
)
for o in objs['Contents']:
    print(o['Key'])


In [ ]:
wrdf = wr.s3.read_csv(
  path='s3://commoncrawl/projects/hyperlinkgraph/cc-main-2024-jun-jul-aug/host/vertices/part-00000-3095dbcf-098e-45c9-a3a7-70c1b93b80fa-c000.txt.gz',
  sep="\t"
)
wrdf.columns=["~id", "~label"]
wrdf

In [ ]:
filtered_wrdf = wrdf[wrdf["~label"].str.endswith('.nz')]
filtered_wrdf

In [ ]:
client = wr.neptune.connect(neptune_hostname, 8182, iam_enabled=True)
# Note this results in Exception ignored ... AttributeError: module 'awswrangler.neptune._gremlin_init' has no attribute 'Client'
# But the data is nevertheless loaded.
wr.neptune.bulk_load(
    client=client,
    df=filtered_wrdf,
    path=f"s3://{work_bucket}/neptune_upload/",
    iam_role=neptune_role,
)
print(wr.neptune.execute_gremlin(client, "g.V().limit(10)"))

In [ ]:
# If you want to retry loading
print(wr.neptune.execute_gremlin(client, "g.V().drop().iterate()"))
print(wr.neptune.execute_gremlin(client, "g.V().limit(10)"))